# SMOTE data

In [ ]:
sm = SMOTE(random_state=42)

In [ ]:
X_res, y_res = sm.fit_resample(combo.drop(['imputed_label'], axis=1), training_labels)

# Initialize arguments for Neural Net

In [ ]:
to_nn = combo.drop(['imputed_label'], axis=1)

In [ ]:
embedding_size = 100
epochs_desired = 20
learning_rate = 0.025
regularization = 0.01
vocabulary_size = len(tfidf.get_feature_names())
algo_optimizer = 'adam'

In [ ]:
model = keras.Sequential([
                        keras.layers.Embedding(input_dim=vocabulary_size, 
                                               output_dim=embedding_size,
                                               embeddings_initializer='normal'
                                              ),
                        keras.layers.GlobalMaxPooling1D(),
                        keras.layers.Dense(200, activation='relu'),
                        keras.layers.Dropout(0.2),
                        keras.layers.Dense(150, activation='relu'),
                        keras.layers.Dropout(0.2),
                        keras.layers.Dense(100, activation='relu'),
                        keras.layers.Dropout(0.2),
                        keras.layers.Dense(len(lab_enc.classes_), activation='softmax')
                      ])

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate, 
                                                  beta_1=0.9, 
                                                  beta_2=0.999, 
                                                  epsilon=1e-6), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [ ]:
model.fit(X_res, y_res, epochs=epochs_desired)

# Use reduced dimensionality (50)

In [ ]:
tsvd1 = TruncatedSVD(n_components=50, random_state=20)

In [ ]:
tsvd1.fit(to_nn)

In [ ]:
tsvd1_ingreds = tsvd1.transform(to_nn)
tsvd1_ingreds.shape

In [ ]:
X_res2, y_res2 = sm.fit_resample(tsvd1_ingreds, training_labels)

In [ ]:
model2 = keras.Sequential([
                        keras.layers.Embedding(input_dim=tsvd1_ingreds.shape[1], 
                                               output_dim=30,
                                               embeddings_initializer='normal'
                                              ),
                        keras.layers.GlobalMaxPooling1D(),
                        keras.layers.Dense(60, activation='relu'),
                        keras.layers.Dropout(0.2),
                        keras.layers.Dense(len(lab_enc.classes_), activation='softmax')
                      ])

In [ ]:
model2.compile(optimizer=keras.optimizers.Adam(learning_rate, 
                                                  beta_1=0.9, 
                                                  beta_2=0.999, 
                                                  epsilon=1e-6), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [ ]:
model2.fit(X_res2, y_res2, epochs=epochs_desired)

# Create target labels

In [ ]:
lab_enc = LabelBinarizer()

In [ ]:
training_labels = lab_enc.fit_transform(combo['imputed_label'])

In [ ]:
lab_enc.classes_

In [ ]:
len(lab_enc.classes_)

In [ ]:
label_index = [np.argmax(label) for label in training_labels]
label_index

In [ ]:
len(label_index)

# Reduce dimensionality (30)

In [ ]:
tsvd2 = TruncatedSVD(n_components=30, random_state=20)

In [ ]:
tsvd2.fit(combo.drop(['imputed_label'], axis=1))
tsvd2_ingreds = tsvd2.transform(combo.drop(['imputed_label'], axis=1))

In [ ]:
tsvd2_ingreds.shape[1]

In [ ]:
model3 = keras.Sequential([
                        keras.layers.Embedding(input_dim=tsvd2_ingreds.shape[1], 
                                               output_dim=tsvd2_ingreds.shape[1],
                                               embeddings_initializer='normal'
                                              ),
                        keras.layers.GlobalMaxPooling1D(),
                        keras.layers.Dense(60, activation='relu'),
                        keras.layers.Dense(60, activation='relu'),
                        keras.layers.Dense(60, activation='relu'),
                        keras.layers.Dense(60, activation='relu'),
                        keras.layers.Dense(len(lab_enc.classes_), activation='softmax')
                      ])

In [ ]:
model3.compile(optimizer=keras.optimizers.Adam(learning_rate, 
                                                  beta_1=0.9, 
                                                  beta_2=0.999, 
                                                  epsilon=1e-6), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [ ]:
X_res2a, y_res2a = sm.fit_resample(tsvd2_ingreds, training_labels)

In [ ]:
model3.fit(X_res2a, y_res2a, epochs=epochs_desired, shuffle=True)

In [ ]:
lab_enc.classes_[2]

In [ ]:
smoted_labels = [lab_enc.classes_[np.argmax(recipe)] for recipe in y_res2a]

In [ ]:
test3=model3.predict(X_res2a)

In [ ]:
test3.shape

In [ ]:
[test3[0] == row for row in test3]

In [ ]:
np.argmax(test3)

# Try Decision Tree instead of Neural Net

In [ ]:
dtc = DecisionTreeClassifier(max_depth=30,random_state=20)

In [ ]:
X_res4, y_res4 = sm.fit_resample(combo.drop(['imputed_label'], axis=1), combo['imputed_label'])   

In [ ]:
dtc = dtc.fit(X_res4, y_res4)

In [ ]:
y_res4_pred = dtc.predict(X_res4)

In [ ]:
print("Accuracy:",sklearn.metrics.accuracy_score(y_res4, y_res4_pred))

In [ ]:
print(sklearn.metrics.confusion_matrix(y_res4, y_res4_pred))

In [ ]:
print(sklearn.metrics.classification_report(y_res4, y_res4_pred))

In [ ]:
test_labeled_cuisine_df['imputed_label']

In [ ]:
test_nlpskl

In [ ]:
test_pred = dtc.predict(test_nlpskl)

In [ ]:
print("Accuracy:",sklearn.metrics.accuracy_score(test_labeled_cuisine_df['imputed_label'], 
                                                 test_pred))
print(sklearn.metrics.confusion_matrix(test_labeled_cuisine_df['imputed_label'], 
                                                 test_pred))
print(sklearn.metrics.classification_report(test_labeled_cuisine_df['imputed_label'], 
                                                 test_pred))

---

In [ ]:
dtc2 = DecisionTreeClassifier(max_depth=200,random_state=20)
X_res5 = combo.drop(['imputed_label'], axis=1)
y_res5 = combo['imputed_label']
dtc2 = dtc2.fit(X_res5, y_res5)
y_res5_pred = dtc.predict(X_res5)
print("Accuracy:",sklearn.metrics.accuracy_score(y_res5, y_res5_pred))
print(sklearn.metrics.confusion_matrix(y_res5, y_res5_pred))
print(sklearn.metrics.classification_report(y_res5, y_res5_pred))

In [ ]:
test_pred2 = dtc2.predict(test_nlpskl)
print("Accuracy:",sklearn.metrics.accuracy_score(test_labeled_cuisine_df['imputed_label'], 
                                                 test_pred2))
print(sklearn.metrics.confusion_matrix(test_labeled_cuisine_df['imputed_label'], 
                                                 test_pred2))
print(sklearn.metrics.classification_report(test_labeled_cuisine_df['imputed_label'], 
                                                 test_pred2))